# Nhận diện Thực thể Được đặt tên (NER)

Notebook này thuộc [Chương trình học AI cho Người mới bắt đầu](http://aka.ms/ai-beginners).

Trong ví dụ này, chúng ta sẽ học cách huấn luyện mô hình NER trên Bộ dữ liệu [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) từ Kaggle. Trước khi tiếp tục, vui lòng tải tệp [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) về thư mục hiện tại.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Chuẩn bị tập dữ liệu

Chúng ta sẽ bắt đầu bằng cách đọc tập dữ liệu vào một dataframe. Nếu bạn muốn tìm hiểu thêm về cách sử dụng Pandas, hãy truy cập [bài học về xử lý dữ liệu](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) trong [Khoa học Dữ liệu cho Người mới bắt đầu](http://aka.ms/datascience-beginners)


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Hãy lấy các thẻ duy nhất và tạo các từ điển tra cứu mà chúng ta có thể sử dụng để chuyển đổi các thẻ thành các số lớp:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Bây giờ chúng ta cần làm điều tương tự với từ vựng. Để đơn giản, chúng ta sẽ tạo từ vựng mà không tính đến tần suất từ; trong thực tế, bạn có thể muốn sử dụng Keras vectorizer và giới hạn số lượng từ.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Chúng ta cần tạo một tập dữ liệu các câu để huấn luyện. Hãy lặp qua tập dữ liệu gốc và tách tất cả các câu riêng lẻ thành `X` (danh sách các từ) và `Y` (danh sách các token):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Để đơn giản, chúng ta sẽ đệm tất cả các câu bằng 0 token đến độ dài tối đa. Trong thực tế, chúng ta có thể muốn sử dụng chiến lược thông minh hơn và chỉ đệm các chuỗi trong một minibatch.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Định nghĩa Mạng Phân loại Token

Chúng ta sẽ sử dụng mạng LSTM hai lớp theo hướng song song để phân loại token. Để áp dụng bộ phân loại dày đặc cho từng đầu ra của lớp LSTM cuối cùng, chúng ta sẽ sử dụng cấu trúc `TimeDistributed`, cấu trúc này sẽ sao chép cùng một lớp dày đặc cho từng đầu ra của LSTM tại mỗi bước:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Lưu ý rằng ở đây chúng ta đang chỉ định rõ `maxlen` cho tập dữ liệu của mình - trong trường hợp muốn mạng có khả năng xử lý các chuỗi có độ dài thay đổi, chúng ta cần phải khéo léo hơn khi định nghĩa mạng.

Bây giờ hãy huấn luyện mô hình. Để tăng tốc, chúng ta sẽ chỉ huấn luyện trong một epoch, nhưng bạn có thể thử huấn luyện trong thời gian dài hơn. Ngoài ra, bạn có thể muốn tách một phần của tập dữ liệu làm tập huấn luyện để quan sát độ chính xác của việc xác thực.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Kiểm tra Kết quả

Bây giờ hãy xem mô hình nhận diện thực thể của chúng ta hoạt động như thế nào trên một câu mẫu:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Kết luận

Ngay cả mô hình LSTM đơn giản cũng cho thấy kết quả hợp lý trong NER. Tuy nhiên, để đạt được kết quả tốt hơn nhiều, bạn có thể muốn sử dụng các mô hình ngôn ngữ lớn đã được huấn luyện trước như BERT. Việc huấn luyện BERT cho NER bằng thư viện Huggingface Transformers được mô tả [tại đây](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
